In [2]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import sys
import glob
import pickle
import copy
import random
import time
import traceback
import shutil

from multiprocessing import Process, Manager

# Config

In [3]:
PROJECT_ROOT = '/workspace/GREAT_ASV_system/'
sys.path.append(PROJECT_ROOT)

In [4]:
OPT_INDEX = '/workspace/DATASET/server9_ssd/STD_VOX1_EGS'
if not os.path.exists(OPT_INDEX):
    os.mkdir(OPT_INDEX)

In [5]:
VOX_TRAIN_DIR = '/workspace/DATASET/server9/voxceleb'
AUG_INFO_DIR = '/workspace/DATASET/server9/STD_musan&rir_info'

In [6]:
INFO_DIR = os.path.join(OPT_INDEX, 'VOX1_INFO')
if not os.path.exists(INFO_DIR):
    os.mkdir(INFO_DIR)

spk2id = os.path.join(INFO_DIR, 'spk2id')
spk2utt_dict = os.path.join(INFO_DIR, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(INFO_DIR, 'spk2utt_train_dict')
spk2utt_val_dict = os.path.join(INFO_DIR, 'spk2utt_val_dict')
spk2utt_train_len = os.path.join(INFO_DIR, 'spk2utt_train_len')
spk2utt_val_len = os.path.join(INFO_DIR, 'spk2utt_val_len')

In [7]:
val_utt_num = 200

# Make spk2utt dict

In [8]:
spk2utt = {}
spk2utt_train = {}
spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

label_dir = glob.glob(VOX_TRAIN_DIR+'/vox1/dev/wav/*')
label = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label):
    with open(spk2id, 'a') as f:
        f.write(str(count)+','+i+'\n')
    spk2utt[count] = glob.glob(os.path.join(VOX_TRAIN_DIR+'/vox1/dev/wav/', i)+'/*/*.wav')

In [9]:
len(spk2utt)

1211

In [10]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

148642


In [11]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)

In [12]:
utt_length = []
for i in spk2utt:
    utt_length.append(len(spk2utt[i]))

In [13]:
utt_length = np.argsort(utt_length)[::-1]

In [14]:
spk2utt_train = copy.deepcopy(spk2utt)
for i in range(val_utt_num):   
    spk2utt_val[utt_length[i]] = [spk2utt[utt_length[i]][0]]
    spk2utt_train[utt_length[i]] = spk2utt[utt_length[i]][1:]

In [15]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

148442


In [16]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [17]:
count = 0
count_list = []
for i in spk2utt_val:
    count += len(spk2utt_val[i])
    count_list.append(len(spk2utt[i]))
print(count)

200


In [18]:
with open(spk2utt_val_dict, 'wb') as handle:
    pickle.dump(spk2utt_val, handle)

## Use multi-process to check read wav

In [19]:
manager = Manager()
spk2utt_train_len_dict_m = manager.list()
eer_m = manager.list()

In [20]:
# err = []
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

In [21]:
spk2utt_train_splitlist = []
num_per_process = len(spk2utt_train) // 10
for i in range(9):
    spk2utt_train_splitlist.append([j for j in range(i*num_per_process, (i+1)*num_per_process)])
    spk2utt_train_len_dict_m.append({})
spk2utt_train_splitlist.append([j for j in range(9*num_per_process, len(spk2utt_train))])
spk2utt_train_len_dict_m.append({})

In [24]:
def read_wav_m(p, spk2utt_train, spk2utt_train_split, spk2utt_train_len_dict_m, eer_m):
    spk2utt_train_len_dict = {}
    for count, i in enumerate(spk2utt_train_split):
        for utt in spk2utt_train[i]:
            try:
                spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
            except Exception:
                eer_m.append([p, utt])
                continue
        print(count)
    spk2utt_train_len_dict_m[p] = spk2utt_train_len_dict

In [25]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

processes = [Process(target = read_wav_m, args = (i, spk2utt_train, spk2utt_train_splitlist[i], spk2utt_train_len_dict_m, eer_m)) for i in range(10)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

0
0
0
0
0
0
0
0
1
1
0
1
1
1
1
1
1
0
2
2
2
2
2
3
1
3
1
3
3
4
4
2
2
2
4
2
4
5
3
6
3
5
3
5
3
2
5
7
4
3
4
3
4
6
8
4
6
9
4
6
7
5
5
10
5
6
8
7
5
7
11
6
7
6
6
8
4
9
8
5
7
9
7
12
7
9
10
5
8
8
10
13
11
8
9
14
6
10
9
6
8
7
15
11
9
12
10
7
11
16
9
17
12
13
10
10
10
18
12
11
11
11
8
12
19
13
11
8
12
13
12
14
9
20
14
10
12
9
13
13
15
15
16
13
14
14
13
16
15
14
15
10
16
11
15
14
17
14
17
11
12
16
21
17
15
12
15
22
17
16
18
13
13
17
18
16
23
18
19
18
17
16
14
18
14
20
18
19
19
24
15
19
20
17
19
21
15
20
25
20
20
16
21
16
21
17
22
22
18
21
18
22
19
21
22
19
19
23
23
17
22
26
23
20
23
20
27
21
18
23
24
21
28
25
24
22
24
20
24
22
25
21
24
29
23
25
19
25
24
26
26
20
25
30
26
25
26
22
26
21
27
31
27
28
27
27
26
22
28
23
27
32
28
23
29
28
27
33
24
28
28
23
29
30
29
34
24
29
30
29
24
25
31
29
30
25
30
26
31
35
32
31
32
26
25
30
31
27
36
26
27
28
33
31
30
29
30
37
34
28
38
31
32
32
33
27
3931

29
32
35
30
32
33
32
33
33
28
31
33
34
34
33
34
36
32
34
37
40
35
29
34
36
34
38
41
39
33
35
35
35
42
36
40
30
35
34

In [26]:
with open(OPT_INDEX+'/spk2utt_train_len_TMP', 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict_m, handle)

In [27]:
spk2utt_train_len_dict = {}
for this_dict in spk2utt_train_len_dict_m:
    for count, i in enumerate(this_dict):
        spk2utt_train_len_dict[i] = this_dict[i]
#         print(count)
print('train_len_num', len(spk2utt_train_len_dict))

train_len_num 148442


In [28]:
err_list = list(eer_m)

In [29]:
err_list

[]

In [30]:
with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

In [ ]:
# spk2utt_train_len_dict = {}

In [ ]:
# err = []
# with open(spk2utt_train_dict, 'rb') as handle:
#     spk2utt_train = pickle.load(handle)

# for i in spk2utt_train:
#     for utt in spk2utt_train[i]:
#         try:
#             spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
#         except Exception:
#             err.append(utt)
#             continue
#     print(i)
            
# with open(spk2utt_train_len, 'wb') as handle:
#     pickle.dump(spk2utt_train_len_dict, handle)

In [31]:
spk2utt_val_len_dict = {}

In [32]:
err = []
with open(spk2utt_val_dict, 'rb') as handle:
    spk2utt_val = pickle.load(handle)

for i in spk2utt_val:
    for utt in spk2utt_val[i]:
        try:
            spk2utt_val_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
        except Exception:
            err.append(utt)
            continue
    print(i)

with open(spk2utt_val_len, 'wb') as handle:
    pickle.dump(spk2utt_val_len_dict, handle)

1150
956
398
163
35
162
963
796
877
728
571
1189
288
1108
1158
1133
1172
395
1014
543
1102
1112
250
505
154
823
758
1040
966
62
1082
303
329
1034
510
1191
871
1179
808
44
102
468
838
326
970
82
619
10
204
769
67
246
172
1141
418
253
637
789
41
16
160
292
308
986
1063
169
20
734
866
884
1056
23
732
1024
1009
108
428
1065
538
429
1042
27
696
390
822
332
907
450
18
341
405
679
821
479
660
1078
830
1130
1174
567
371
563
820
1010
795
120
673
147
304
860
131
124
156
921
1041
161
731
431
843
804
1142
356
924
294
219
579
634
365
586
14
259
989
1100
179
1080
847
524
422
1067
239
1028
129
200
178
1204
712
954
652
93
666
900
1036
740
1160
1114
145
1086
339
1155
1022
578
208
6
812
612
481
555
649
137
419
87
30
978
521
1122
78
617
776
11
467
224
258
76
281
442
982
381
736
507
201
243
539
642
66
46
864
623
1101
475
337


In [34]:
err

[]

# Make train data

In [35]:
from preprocessing.Utils.preprocessing_3type import ThreeTypes_IterableDataset

In [36]:
data_dir_dict = {}

data_dir_dict['spk2utt_train_dict'] = os.path.join(INFO_DIR, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(AUG_INFO_DIR, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(AUG_INFO_DIR, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(AUG_INFO_DIR, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(AUG_INFO_DIR, 'rir_dict')

In [37]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(INFO_DIR, 'spk2utt_train_len')
data_len_dict['music_len'] = os.path.join(AUG_INFO_DIR, 'music_len')
data_len_dict['noise_len'] = os.path.join(AUG_INFO_DIR, 'noise_len')
data_len_dict['babble_len'] = os.path.join(AUG_INFO_DIR, 'babble_len')

## Multi episode process

In [38]:
PROCESS = 1500
RUNNING_P = 10

MFCC_O = os.path.join(OPT_INDEX, 'TRAIN_VOX2_MFCC')
LOGMELFB_O = os.path.join(OPT_INDEX, 'TRAIN_VOX2_FB')
RAWWAV_O = os.path.join(OPT_INDEX, 'TRAIN_VOX2_RAW')
EER_LOG = os.path.join(OPT_INDEX, 'EER_LOG')
G_LOG = os.path.join(OPT_INDEX, 'G_LOG')

In [39]:
if not os.path.exists(MFCC_O):
    os.mkdir(MFCC_O)
else:
    shutil.rmtree(MFCC_O)
    
if not os.path.exists(LOGMELFB_O):
    os.mkdir(LOGMELFB_O)
else:
    shutil.rmtree(LOGMELFB_O)
    
if not os.path.exists(RAWWAV_O):
    os.mkdir(RAWWAV_O)
else:
    shutil.rmtree(RAWWAV_O)

with open(EER_LOG, 'w') as f:
    pass

with open(G_LOG, 'w') as f:
    pass

In [40]:
config = {}

config['sr'] = 16000
config['repeats'] = 1
config['batch_size'] = 1

In [41]:
def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (rw, fb, mfcc, label) in enumerate(dataset):
            file_name = str(i)+str('_')+str(count)
            with open(os.path.join(RAWWAV_O, file_name), 'wb') as handle:
                pickle.dump((rw.astype(np.float16), label.astype(np.int16)), handle)
                
            with open(os.path.join(LOGMELFB_O, file_name), 'wb') as handle:
                pickle.dump((fb.astype(np.float16), label.astype(np.int16)), handle)
                
            with open(os.path.join(MFCC_O, file_name), 'wb') as handle:
                pickle.dump((mfcc.astype(np.float16), label.astype(np.int16)), handle)
            
            with open(G_LOG, 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            
    except Exception:
        with open(EER_LOG, 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = ThreeTypes_IterableDataset(config, data_dir_dict, data_len_dict)
dataset.noise_data_preload()

p_list = []
for i in range(PROCESS // RUNNING_P):
    p_list.append([j for j in range(i*RUNNING_P, i*RUNNING_P+RUNNING_P)])
if (PROCESS % RUNNING_P) != 0:
    p_list.append([j for j in range((i+1)*RUNNING_P, PROCESS)])

print('episode', len(p_list))
print('last episode', p_list[-1])

start_time = time.time()
for ps in p_list:
    print(ps[0])
    processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in ps]
    [p.start() for p in processes]
    joined = [p.join() for p in processes]

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
episode 150
last episode [1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490


In [ ]:
# # normal
# config = {}

# config['sr'] = 16000
# config['repeats'] = 150
# config['batch_size'] = 1
# config['extended_prefectch'] = 1.0
# config['running_p'] = 10
# config['out_dir'] = '/Lun0/zhiyong/dataset/test_1/'
# config['err_log'] = '/Lun0/zhiyong/dataset/tmp_data_errlog'

# def iter_data_preload(dataset, i):
#     dataset.get_random_list()
#     try:
#         start_time = time.time()
#         for count, (data, label) in enumerate(dataset):
#             with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
#                 pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
# #             print(time.time()-start_time)
#             with open(config['err_log'], 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
#             start_time = time.time()
            
#             if ((count+1)%10000) == 0:
#                 print((count+1)//10000)
#     except Exception:
#         with open(config['err_log'], 'a') as f:
#             traceback.print_exc(file=f)
#         raise Exception

# dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# # dataset.noise_data_preload2mem()
# dataset.noise_data_preload()

# processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(config['running_p'])]
# start_time = time.time()
# [p.start() for p in processes]
# joined = [p.join() for p in processes]
# print(time.time()-start_time)

# Make train index

In [42]:
train_mfcc_dir = os.path.join(INFO_DIR, 'train_mfcc.csv')
train_fb_dir = os.path.join(INFO_DIR, 'train_fb.csv')
train_rw_dir = os.path.join(INFO_DIR, 'train_raw.csv')

In [43]:
mfcc_len = glob.glob(MFCC_O+'/*')
fb_len = glob.glob(LOGMELFB_O+'/*')
raw_len = glob.glob(RAWWAV_O+'/*')

In [44]:
print('mfcc', len(mfcc_len))
print('fb', len(fb_len))
print('raw', len(raw_len))

mfcc 1816500
fb 1816500
raw 1816500


In [45]:
count = 0
with open(train_mfcc_dir, 'w') as f:
    for path in mfcc_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('mfcc', count)

count = 0
with open(train_fb_dir, 'w') as f:
    for path in fb_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('fb', count)

count = 0
with open(train_rw_dir, 'w') as f:
    for path in raw_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('raw', count)

mfcc 1816500
fb 1816500
raw 1816500


In [46]:
1211 * 1500

1816500

# Make validation data

In [48]:
VAL_MFCC_O = os.path.join(OPT_INDEX, 'VAL_MFCC')
VAL_LOGMELFB_O = os.path.join(OPT_INDEX, 'VAL_FB')
VAL_RAWWAV_O = os.path.join(OPT_INDEX, 'VAL_RAW')

In [49]:
data_dir_dict = {}

data_dir_dict['spk2utt_train_dict'] = os.path.join(INFO_DIR, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(AUG_INFO_DIR, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(AUG_INFO_DIR, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(AUG_INFO_DIR, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(AUG_INFO_DIR, 'rir_dict')

In [50]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(INFO_DIR, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(AUG_INFO_DIR, 'music_len')
data_len_dict['noise_len'] = os.path.join(AUG_INFO_DIR, 'noise_len')
data_len_dict['babble_len'] = os.path.join(AUG_INFO_DIR, 'babble_len')

In [51]:
if not os.path.exists(VAL_MFCC_O):
    os.mkdir(VAL_MFCC_O)
else:
    shutil.rmtree(VAL_MFCC_O)
    os.mkdir(VAL_MFCC_O)
    
if not os.path.exists(VAL_LOGMELFB_O):
    os.mkdir(VAL_LOGMELFB_O)
else:
    shutil.rmtree(VAL_LOGMELFB_O)
    os.mkdir(VAL_LOGMELFB_O)
    
if not os.path.exists(VAL_RAWWAV_O):
    os.mkdir(VAL_RAWWAV_O)
else:
    shutil.rmtree(VAL_RAWWAV_O)
    os.mkdir(VAL_RAWWAV_O)

In [52]:
config = {}

config['sr'] = 16000
config['repeats'] = 5
config['batch_size'] = 1

In [53]:
def iter_data_preload(dataset, i):
    dataset.get_random_list()
    for count, (rw, fb, mfcc, label) in enumerate(dataset):
        file_name = str(i)+str('_')+str(count)
        with open(os.path.join(VAL_RAWWAV_O, file_name), 'wb') as handle:
            pickle.dump((rw.astype(np.float16), label.astype(np.int16)), handle)

        with open(os.path.join(VAL_LOGMELFB_O, file_name), 'wb') as handle:
            pickle.dump((fb.astype(np.float16), label.astype(np.int16)), handle)

        with open(os.path.join(VAL_MFCC_O, file_name), 'wb') as handle:
            pickle.dump((mfcc.astype(np.float16), label.astype(np.int16)), handle)
        start_time = time.time()

dataset = ThreeTypes_IterableDataset(config, data_dir_dict, data_len_dict)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400


# Make validation index

In [55]:
val_mfcc_dir = os.path.join(INFO_DIR, 'val_mfcc.csv')
val_fb_dir = os.path.join(INFO_DIR, 'val_fb.csv')
val_rw_dir = os.path.join(INFO_DIR, 'val_raw.csv')

In [56]:
mfcc_len = glob.glob(VAL_MFCC_O+'/*')
fb_len = glob.glob(VAL_LOGMELFB_O+'/*')
raw_len = glob.glob(VAL_RAWWAV_O+'/*')

In [57]:
print('mfcc', len(mfcc_len))
print('fb', len(fb_len))
print('raw', len(raw_len))

mfcc 1000
fb 1000
raw 1000


In [58]:
count = 0
with open(val_mfcc_dir, 'w') as f:
    for path in mfcc_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('mfcc', count)

count = 0
with open(val_fb_dir, 'w') as f:
    for path in fb_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('fb', count)

count = 0
with open(val_rw_dir, 'w') as f:
    for path in raw_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('raw', count)

mfcc 1000
fb 1000
raw 1000
